In [1]:
import re
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

In [2]:
def select_gpu(N):
    gpus = tf.config.experimental.list_physical_devices('GPU')
    print(gpus)
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            tf.config.experimental.set_visible_devices(gpus[N], 'GPU')
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            print(e)    

In [3]:
select_gpu(0)
epochs = 512
batch_size = 1024
model_name = 'bert-chinese'
embedding_path = '/home/Danny/pretrain_model/{}'.format(model_name)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')]
3 Physical GPUs, 1 Logical GPUs


In [4]:
# 不用儲存article_id, 轉成list
def training_txt_to_list(path):
    with open(path, 'r') as f:
        txt = str(f.read())
    txt_list = txt.split('\n')
    text_label_list = list()
    tmp = list()
    for line in txt_list:
        if line == '--------------------':
            text_label_list.append(tmp)
            tmp = list()
            continue
        if line == '':
            continue
        tmp.append(line)
    x = list()
    y = list()
    for text_label in text_label_list:
        text = text_label[0]
        label = text_label[2:]
        label_list = ['O' for i in range(len(text))]
        for i in label:
            entity = i.split('\t')
            if int(entity[1]) > int(entity[2]):
                continue
            b = int(entity[1])
            label_list[b] = 'B-{}'.format(entity[-1])
            for j in range(int(entity[1])+1, int(entity[2])):
                label_list[j] = 'I-{}'.format(entity[-1])
        for i, j in enumerate(text):
            if j == '，' or j == '。' or j == '？':
                label_list[i] = j
                
        text_list = re.split('\uff0c|\u3002|\uff1f', text)
        for sentence in text_list:
            x.append([i for i in sentence])
        x = x[:-1]
            
        sentence = list()
        for i in label_list:
            if i == '，' or i == '。' or i == '？':
                y.append(sentence)
                sentence = list()
            else:
                sentence.append(i)
                
    return [x, y]

In [5]:
x1, y1 = training_txt_to_list('/home/Danny/AI-CUP-2020/datasets/stage1/SampleData_deid.txt')

In [6]:
x2, y2 = training_txt_to_list('/home/Danny/AI-CUP-2020/datasets/stage2/train_1_update.txt')

In [7]:
x4, y4 = training_txt_to_list('/home/Danny/AI-CUP-2020/datasets/stage4/train_2.txt')

In [8]:
x = x1 + x2 + x4
y = y1 + y2 + y4
print(len(x))
print(len(y))

76162
76162


In [9]:
df = pd.DataFrame({'x': x, 'y': y})
df

,x,y
0,"[醫, 師, ：, 你, 有, 做, 超, 音, 波, 嘛]","[O, O, O, O, O, O, O, O, O, O]"
1,"[那, 我, 們, 來, 看, 報, 告]","[O, O, O, O, O, O, O]"
2,"[有, 些, 部, 分, 有, 紅, 字, 耶]","[O, O, O, O, O, O, O, O]"
3,"[民, 眾, ：, 紅, 字, 是, 甚, 麼, 意, 思]","[O, O, O, O, O, O, O, O, O, O]"
4,"[醫, 師, ：, 就, 是, 肝, 功, 能, 有, 比, 較, 高]","[O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...
76157,"[醫, 師, ：, 他, 有, 在, 騎, 腳, 踏, 車]","[O, O, O, O, O, O, O, O, O, O]"
76158,"[民, 眾, ：, 恩]","[O, O, O, O]"
76159,"[醫, 師, ：, 騎, 腳, 踏, 車, 可, 以]","[O, O, O, O, O, O, O, O, O]"
76160,"[小, 美]","[B-name, I-name]"


In [10]:
clinical_event = pd.read_csv('/home/Danny/AI-CUP-2020/datasets/augmentation/clinical_event.csv')
contact = pd.read_csv('/home/Danny/AI-CUP-2020/datasets/augmentation/contact.csv')
education = pd.read_csv('/home/Danny/AI-CUP-2020/datasets/augmentation/education.csv')
family = pd.read_csv('/home/Danny/AI-CUP-2020/datasets/augmentation/family.csv')
profession = pd.read_csv('/home/Danny/AI-CUP-2020/datasets/augmentation/profession.csv')

In [11]:
df = df.append(clinical_event)
df = df.append(contact)
df = df.append(education)
df = df.append(family)
df = df.append(profession)
df

,x,y
0,"[醫, 師, ：, 你, 有, 做, 超, 音, 波, 嘛]","[O, O, O, O, O, O, O, O, O, O]"
1,"[那, 我, 們, 來, 看, 報, 告]","[O, O, O, O, O, O, O]"
2,"[有, 些, 部, 分, 有, 紅, 字, 耶]","[O, O, O, O, O, O, O, O]"
3,"[民, 眾, ：, 紅, 字, 是, 甚, 麼, 意, 思]","[O, O, O, O, O, O, O, O, O, O]"
4,"[醫, 師, ：, 就, 是, 肝, 功, 能, 有, 比, 較, 高]","[O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...
495,"['那', '個', '時', '候', '是', '白', '天', '釣', '具', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-profess..."
496,"['因', '為', '我', '是', '還', '是', '網', '球', '球', ...","['O', 'O', 'O', 'O', 'O', 'O', 'B-profession',..."
497,"['醫', '師', '：', '你', '是', '採', '椰', '子', '工', ...","['O', 'O', 'O', 'O', 'O', 'B-profession', 'I-p..."
498,"['安', '養', '院', '工', '作', '人', '員']","['B-profession', 'I-profession', 'I-profession..."


In [13]:
df['x'] = df['x'].apply(lambda row : str(row))
df['y'] = df['y'].apply(lambda row : str(row))
# df = df.drop_duplicates()
df['x'] = df['x'].apply(lambda row : eval(row))
df['y'] = df['y'].apply(lambda row : eval(row))
df

,x,y
0,"[醫, 師, ：, 你, 有, 做, 超, 音, 波, 嘛]","[O, O, O, O, O, O, O, O, O, O]"
1,"[那, 我, 們, 來, 看, 報, 告]","[O, O, O, O, O, O, O]"
2,"[有, 些, 部, 分, 有, 紅, 字, 耶]","[O, O, O, O, O, O, O, O]"
3,"[民, 眾, ：, 紅, 字, 是, 甚, 麼, 意, 思]","[O, O, O, O, O, O, O, O, O, O]"
4,"[醫, 師, ：, 就, 是, 肝, 功, 能, 有, 比, 較, 高]","[O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...
495,"[那, 個, 時, 候, 是, 白, 天, 釣, 具, 製, 造, 工, 人]","[O, O, O, O, O, O, O, B-profession, I-professi..."
496,"[因, 為, 我, 是, 還, 是, 網, 球, 球, 員, 嘛]","[O, O, O, O, O, O, B-profession, I-profession,..."
497,"[醫, 師, ：, 你, 是, 採, 椰, 子, 工, 是, 不, 是]","[O, O, O, O, O, B-profession, I-profession, I-..."
498,"[安, 養, 院, 工, 作, 人, 員]","[B-profession, I-profession, I-profession, I-p..."


In [14]:
x = df['x'].to_list()
y = df['y'].to_list()

In [15]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
print(len(train_x), len(train_y))
print(len(valid_x), len(valid_y))
print(len(test_x), len(test_y))

50151 50151
12538 12538
15673 15673


In [ ]:
%%time
import kashgari
from kashgari.tasks.labeling import BiLSTM_CRF_Model
from kashgari.embeddings import BertEmbedding
kashgari.config.use_cudnn_cell = True
bert_embedding = BertEmbedding(embedding_path, 
                           sequence_length='auto',
                           trainable=True,
                           task='kashgari.LABELING', 
                          )
model = BiLSTM_CRF_Model(bert_embedding)
history = model.fit(train_x,
                    train_y,
                    valid_x,
                    valid_y,
                    epochs=epochs,
                    batch_size=batch_size,
                   )

2020-12-17 17:05:43,369 [DEBUG] kashgari - ------------------------------------------------
2020-12-17 17:05:43,370 [DEBUG] kashgari - Loaded transformer model's vocab
2020-12-17 17:05:43,370 [DEBUG] kashgari - config_path       : /home/Danny/pretrain_model/bert-chinese/bert_config.json
2020-12-17 17:05:43,370 [DEBUG] kashgari - vocab_path      : /home/Danny/pretrain_model/bert-chinese/vocab.txt
2020-12-17 17:05:43,371 [DEBUG] kashgari - checkpoint_path : /home/Danny/pretrain_model/bert-chinese/bert_model.ckpt
2020-12-17 17:05:43,371 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[

Epoch 1/512
48/48 [==============================] - 38s 784ms/step - loss: 2.2985 - accuracy: 0.9125 - val_loss: 25.5652 - val_accuracy: 0.9695
Epoch 2/512
48/48 [==============================] - 35s 726ms/step - loss: 0.7009 - accuracy: 0.9698 - val_loss: 25.4548 - val_accuracy: 0.9762
Epoch 3/512
48/48 [==============================] - 35s 728ms/step - loss: 0.4870 - accuracy: 0.9746 - val_loss: 25.3767 - val_accuracy: 0.9798
Epoch 4/512
48/48 [==============================] - 35s 732ms/step - loss: 0.3694 - accuracy: 0.9782 - val_loss: 25.2754 - val_accuracy: 0.9835
Epoch 5/512
48/48 [==============================] - 35s 735ms/step - loss: 0.2975 - accuracy: 0.9819 - val_loss: 25.1399 - val_accuracy: 0.9857
Epoch 6/512
48/48 [==============================] - 35s 735ms/step - loss: 0.2513 - accuracy: 0.9844 - val_loss: 25.0910 - val_accuracy: 0.9875
Epoch 7/512
48/48 [==============================] - 35s 736ms/step - loss: 0.2202 - accuracy: 0.9859 - val_loss: 25.0104 - val_ac

48/48 [==============================] - 36s 742ms/step - loss: 0.0225 - accuracy: 0.9982 - val_loss: 23.3312 - val_accuracy: 0.9895
Epoch 58/512
48/48 [==============================] - 36s 743ms/step - loss: 0.0234 - accuracy: 0.9981 - val_loss: 23.3117 - val_accuracy: 0.9896
Epoch 59/512
48/48 [==============================] - 36s 742ms/step - loss: 0.0218 - accuracy: 0.9981 - val_loss: 23.3084 - val_accuracy: 0.9895
Epoch 60/512
48/48 [==============================] - 36s 743ms/step - loss: 0.0219 - accuracy: 0.9982 - val_loss: 23.2775 - val_accuracy: 0.9893
Epoch 61/512
48/48 [==============================] - 36s 742ms/step - loss: 0.0207 - accuracy: 0.9982 - val_loss: 23.2728 - val_accuracy: 0.9880
Epoch 62/512
48/48 [==============================] - 36s 742ms/step - loss: 0.0205 - accuracy: 0.9983 - val_loss: 23.2355 - val_accuracy: 0.9879
Epoch 63/512
48/48 [==============================] - 36s 742ms/step - loss: 0.0201 - accuracy: 0.9982 - val_loss: 23.2063 - val_accuracy

48/48 [==============================] - 36s 742ms/step - loss: 0.0106 - accuracy: 0.9988 - val_loss: 22.0774 - val_accuracy: 0.0520
Epoch 114/512
48/48 [==============================] - 36s 742ms/step - loss: 0.0110 - accuracy: 0.9987 - val_loss: 22.0382 - val_accuracy: 0.0516
Epoch 115/512
48/48 [==============================] - 36s 743ms/step - loss: 0.0100 - accuracy: 0.9989 - val_loss: 22.0412 - val_accuracy: 0.0518
Epoch 116/512
48/48 [==============================] - 36s 742ms/step - loss: 0.0100 - accuracy: 0.9989 - val_loss: 22.0029 - val_accuracy: 0.0481
Epoch 117/512
48/48 [==============================] - 36s 741ms/step - loss: 0.0106 - accuracy: 0.9988 - val_loss: 21.9545 - val_accuracy: 0.0480
Epoch 118/512
48/48 [==============================] - 36s 741ms/step - loss: 0.0106 - accuracy: 0.9988 - val_loss: 21.9468 - val_accuracy: 0.0411
Epoch 119/512
48/48 [==============================] - 36s 747ms/step - loss: 0.0093 - accuracy: 0.9990 - val_loss: 21.9431 - val_ac

Epoch 169/512
48/48 [==============================] - 36s 742ms/step - loss: 0.0078 - accuracy: 0.9989 - val_loss: 20.4862 - val_accuracy: 0.0291
Epoch 170/512
48/48 [==============================] - 36s 742ms/step - loss: 0.0067 - accuracy: 0.9990 - val_loss: 20.4672 - val_accuracy: 0.0296
Epoch 171/512
48/48 [==============================] - 36s 741ms/step - loss: 0.0073 - accuracy: 0.9990 - val_loss: 20.4264 - val_accuracy: 0.0298
Epoch 172/512
48/48 [==============================] - 36s 742ms/step - loss: 0.0074 - accuracy: 0.9989 - val_loss: 20.3957 - val_accuracy: 0.0298
Epoch 173/512
48/48 [==============================] - 36s 743ms/step - loss: 0.0071 - accuracy: 0.9990 - val_loss: 20.3505 - val_accuracy: 0.0296
Epoch 174/512
48/48 [==============================] - 36s 742ms/step - loss: 0.0072 - accuracy: 0.9989 - val_loss: 20.3417 - val_accuracy: 0.0295
Epoch 175/512
48/48 [==============================] - 36s 743ms/step - loss: 0.0075 - accuracy: 0.9990 - val_loss: 20

Epoch 225/512
48/48 [==============================] - 36s 743ms/step - loss: 0.0066 - accuracy: 0.9989 - val_loss: 18.7755 - val_accuracy: 0.0211
Epoch 226/512
48/48 [==============================] - 36s 742ms/step - loss: 0.0064 - accuracy: 0.9990 - val_loss: 18.7604 - val_accuracy: 0.0212
Epoch 227/512
48/48 [==============================] - 36s 741ms/step - loss: 0.0058 - accuracy: 0.9991 - val_loss: 18.7487 - val_accuracy: 0.0214
Epoch 228/512
48/48 [==============================] - 36s 742ms/step - loss: 0.0064 - accuracy: 0.9989 - val_loss: 18.7117 - val_accuracy: 0.0208
Epoch 229/512
48/48 [==============================] - 36s 744ms/step - loss: 0.0070 - accuracy: 0.9988 - val_loss: 18.6875 - val_accuracy: 0.0212
Epoch 230/512
48/48 [==============================] - 36s 744ms/step - loss: 0.0067 - accuracy: 0.9990 - val_loss: 18.6762 - val_accuracy: 0.0215
Epoch 231/512
48/48 [==============================] - 36s 743ms/step - loss: 0.0066 - accuracy: 0.9990 - val_loss: 18

Epoch 281/512
48/48 [==============================] - 36s 740ms/step - loss: 0.0052 - accuracy: 0.9991 - val_loss: 17.5833 - val_accuracy: 0.0165
Epoch 282/512
48/48 [==============================] - 36s 741ms/step - loss: 0.0055 - accuracy: 0.9991 - val_loss: 17.6015 - val_accuracy: 0.0160
Epoch 283/512
48/48 [==============================] - 36s 740ms/step - loss: 0.0057 - accuracy: 0.9990 - val_loss: 17.5745 - val_accuracy: 0.0166
Epoch 284/512
48/48 [==============================] - 36s 741ms/step - loss: 0.0060 - accuracy: 0.9990 - val_loss: 17.5260 - val_accuracy: 0.0169
Epoch 285/512
48/48 [==============================] - 36s 740ms/step - loss: 0.0057 - accuracy: 0.9991 - val_loss: 17.4920 - val_accuracy: 0.0168
Epoch 286/512
48/48 [==============================] - 36s 741ms/step - loss: 0.0056 - accuracy: 0.9990 - val_loss: 17.4934 - val_accuracy: 0.0168
Epoch 287/512
48/48 [==============================] - 36s 740ms/step - loss: 0.0048 - accuracy: 0.9991 - val_loss: 17

Epoch 337/512
48/48 [==============================] - 35s 739ms/step - loss: 0.0052 - accuracy: 0.9991 - val_loss: 16.7884 - val_accuracy: 0.0130
Epoch 338/512
48/48 [==============================] - 35s 739ms/step - loss: 0.0052 - accuracy: 0.9990 - val_loss: 16.7563 - val_accuracy: 0.0132
Epoch 339/512
48/48 [==============================] - 36s 740ms/step - loss: 0.0047 - accuracy: 0.9991 - val_loss: 16.7569 - val_accuracy: 0.0134
Epoch 340/512
48/48 [==============================] - 36s 740ms/step - loss: 0.0049 - accuracy: 0.9991 - val_loss: 16.7470 - val_accuracy: 0.0131
Epoch 341/512
48/48 [==============================] - 36s 741ms/step - loss: 0.0050 - accuracy: 0.9991 - val_loss: 16.7335 - val_accuracy: 0.0129
Epoch 342/512
48/48 [==============================] - 36s 740ms/step - loss: 0.0051 - accuracy: 0.9991 - val_loss: 16.7028 - val_accuracy: 0.0131
Epoch 343/512
48/48 [==============================] - 36s 741ms/step - loss: 0.0050 - accuracy: 0.9991 - val_loss: 16

Epoch 393/512
48/48 [==============================] - 36s 741ms/step - loss: 0.0043 - accuracy: 0.9991 - val_loss: 16.2136 - val_accuracy: 0.0106
Epoch 394/512
48/48 [==============================] - 36s 740ms/step - loss: 0.0042 - accuracy: 0.9992 - val_loss: 16.1758 - val_accuracy: 0.0105
Epoch 395/512
48/48 [==============================] - 36s 741ms/step - loss: 0.0050 - accuracy: 0.9990 - val_loss: 16.1830 - val_accuracy: 0.0104
Epoch 396/512
48/48 [==============================] - 36s 741ms/step - loss: 0.0046 - accuracy: 0.9990 - val_loss: 16.1895 - val_accuracy: 0.0103
Epoch 397/512
48/48 [==============================] - 36s 740ms/step - loss: 0.0045 - accuracy: 0.9991 - val_loss: 16.1558 - val_accuracy: 0.0102
Epoch 398/512
48/48 [==============================] - 36s 740ms/step - loss: 0.0042 - accuracy: 0.9991 - val_loss: 16.1210 - val_accuracy: 0.0101
Epoch 399/512
48/48 [==============================] - 36s 740ms/step - loss: 0.0045 - accuracy: 0.9991 - val_loss: 16

In [ ]:
model.evaluate(test_x, test_y)
model_path = 'model/duplicates_augmentation_{}_epoch_{}_batch_{}'.format(model_name, epochs, batch_size)
model.save(model_path)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['accuracy', 'val_accuracy', 'loss', 'val_loss'])
plt.show()

In [ ]:
# 需要儲存article_id, 轉成dataframe
def predicting_txt_to_dataframe(path):    
    with open(path, 'r') as f:
        txt = str(f.read())
        txt_list = txt.split('\n')
    row_list = list()
    tmp_list = list()
    for row in txt_list:
        if row == '--------------------':
            tmp_list[0] = tmp_list[0].replace('article_id:', '')
            tmp_list[0] = int(tmp_list[0])
            row_list.append(tmp_list)
            tmp_list = list()
            continue
        if len(row) > 0:
            tmp_list.append(row)
    df = pd.DataFrame(row_list, columns=['article_id','text'])
    return df

In [ ]:
def predict_NER(text):
    x_list = list()    
    text_list = re.split('\uff0c|\u3002|\uff1f', text)
    for article in text_list:
        x_list.append([i for i in article])
    y_list_list = model.predict(x_list)
    y_list = list()
    for sentence in y_list_list:
        for word in sentence:
            y_list.append(word)
        y_list.append('O') # append(，。？)
    y_list = y_list[:-1]
    return y_list

In [ ]:
def output_NER(article_id, text, y_list):
    output_str = str()
    flag = False
    for i, j in enumerate(y_list):
        if j != 'O':
            if j[0] == 'B':
                start_position = i
                entity_type = j.split('-')[-1]
                flag = True
        elif j == 'O' and flag == True:
            end_position = i
            flag = False
            entity_text = text[start_position: end_position]
            entity = '{}\t{}\t{}\t{}\t{}\n'.format(article_id, start_position, end_position, entity_text, entity_type)  
            output_str += entity
    return output_str

In [ ]:
df = predicting_txt_to_dataframe('/home/Danny/AI-CUP-2020/datasets/stage5/test.txt')
output_str = "article_id\tstart_position\tend_position\tentity_text\tentity_type\n"

for article_id, text in zip(df['article_id'], df['text']):
    x_list = [word for word in text]
    y_list = predict_NER(text)
    output_str += output_NER(article_id, text, y_list)

In [ ]:
output_path = 'output/duplicates_augmentation_{}_epoch_{}_batch_{}.tsv'.format(model_name, epochs, batch_size)
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(output_str)

In [ ]:
df = pd.read_csv(output_path, sep='\t')
df